In [17]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import scipy.io
from sklearn import linear_model
from sklearn import model_selection
import eknn 
from eknn import exclusive_lasso, EkNN_C,EkNN_R
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split

#### Test on LSVT Voice rehabitation dataset 

In [18]:
lsvt_predictors = pd.read_excel('lsvt/LSVT_voice_rehabilitation.xlsx')
lsvt_predictors.shape

(126, 310)

In [19]:
lsvt_response = pd.read_excel('lsvt/LSVT_voice_rehabilitation.xlsx',sheet_name=1)
lsvt_response.shape

(126, 1)

In [20]:
data = pd.concat([lsvt_predictors, lsvt_response], axis=1, join='outer')
data

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,...,det_TKEO_std4_2_coef,det_TKEO_std4_3_coef,det_TKEO_std4_4_coef,det_TKEO_std4_5_coef,det_TKEO_std4_6_coef,det_TKEO_std4_7_coef,det_TKEO_std4_8_coef,det_TKEO_std4_9_coef,det_TKEO_std4_10_coef,"Binary class 1=acceptable, 2=unacceptable"
0,0.088112,0.041697,0.000480,-3.723304e-06,0.000422,2.458381,6.332164e-07,47.021079,1366.430390,-7.103323,...,7.088978,19.753255,54.335046,145.528630,375.097397,921.296579,2137.079844,4697.131077,9931.208257,1
1,0.161798,0.057364,0.000677,5.466365e-06,0.000206,2.592066,7.228518e-07,93.557936,2582.922776,-23.284761,...,7.977363,22.203504,60.993338,163.560972,421.010306,1036.092589,2404.072562,5284.082128,11165.095662,2
2,0.554508,0.642913,0.007576,-7.443871e-07,0.006488,12.691326,6.946246e-04,52.988422,466.682635,-45.308680,...,5.078616,14.135923,38.641654,103.466808,264.654626,649.657090,1507.384591,3315.804236,6974.600636,2
3,0.031089,0.027108,0.000314,-2.214722e-07,0.000216,0.754288,1.868647e-07,13.982754,417.217249,-1.207741,...,5.610448,15.626164,42.943275,115.014975,296.320795,728.284936,1689.586636,3713.818933,7851.139360,1
4,0.076177,0.039071,0.000302,2.732106e-05,0.001102,1.270034,4.918186e-05,56.373996,1608.317410,-3.491990,...,6.902199,19.117609,52.715873,141.113865,363.511021,893.246151,2071.625622,4554.204815,9623.566242,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0.116214,0.070546,0.000837,2.765074e-06,0.000333,1.890006,9.844257e-07,34.083311,896.672947,-10.446397,...,6.527285,18.210931,50.085484,134.123291,345.396264,850.942761,1973.383824,4336.099395,9158.984652,2
122,0.700258,0.334397,0.003959,8.297261e-06,0.001516,4.557797,1.581899e-05,104.648435,1583.166169,-97.281717,...,7.160142,20.081389,55.178858,147.584708,379.897760,935.982559,2166.960428,4769.956102,10067.750435,2
123,0.072635,0.050743,0.000597,-5.277518e-06,0.000434,6.984651,4.993260e-07,21.859427,625.288493,-4.116001,...,6.141876,17.095325,46.893015,125.687344,323.728298,795.715774,1845.609006,4056.256338,8583.121863,1
124,0.111362,0.054237,0.000646,-1.546671e-06,0.000277,1.935398,4.398078e-07,47.870508,1367.843467,-9.373059,...,7.125849,19.849347,54.508453,146.094711,376.377835,926.435019,2147.499571,4717.270683,9966.759379,2


In [21]:
train_set, test_set = train_test_split(data, test_size=0.3)

In [22]:
train_set = train_set.reset_index(drop=True)

In [23]:
test_set = test_set.reset_index(drop=True)

In [24]:
train_set.shape, data.shape
predictors= train_set.columns

##### Procedures for the Exclusive Lasso KNN classifier
- Extract X and Y matrix where X (pxn) are compressed training predictors and label values. Y(pxn) are compressed training predictors and labels values. Label values are the last row of the 2 matrices. 

- Extract `x_labels` and `y_true_labels`. `x_labels` vector is the same as `y_train` in ususal estimators. `y_true_labels` is vector is pass into the score function. Reformat the labels to 0...m if the original ;labels are not formatted in this way. Seperate the labels from the predictors matrix for both `Y` and `X` vectors. 

- To match `sklearn` API, transpose `X` before pass into `.fit` method.

- Implement `GridSearchCV` to find best parameters. Then pass in `Y` and predict the labels for `Y`. Accuracy is provdided for the `score` function of the estimator. 

In [25]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
train_cols = train_set.shape[1] 
train_rows = train_set.shape[0]
X = np.zeros((train_cols, train_rows))
for i,col in enumerate(predictors):
    for j in range(train_rows):
        X[i,j] = train_set[col][j]

In [26]:
# generate test sample matrix Y
# matrix rows = col number
# matrix column = row number
# y = pxn matrix where n = number of test samples 
test_cols = test_set.shape[1]  
test_rows = test_set.shape[0]
Y = np.zeros((test_cols, test_rows))
for i,col in enumerate(predictors):
    for j in range(test_rows):
        Y[i,j] = train_set[col][j]

In [27]:
x_labels = X[-1]
y_true_label = Y[-1]

In [28]:
X = X[0:X.shape[0]-1, :]
Y = Y[0:Y.shape[0]-1,:]
X_transposed = np.transpose(X)

In [333]:
# from sklearn.base import BaseEstimator, ClassifierMixin
# from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
# from sklearn.utils.multiclass import unique_labels
# class ExclusiveLassoKNNClassifier(BaseEstimator, ClassifierMixin):
#     def __init__(self, lambda_=1.0, group_num=1, k=3):
#         self.lambda_ = lambda_
#         self.group_num = group_num
#         self.k = k
    
#     def fit(self, X, x_labels):
        
#         # X passed in have nxp dimension
#         self.group_vect = self.group_encode(X) 
#         self.X = np.transpose(X) # X_train
#         self.x_labels = x_labels # y_train 
    
#         class_names = [i+1 for i in range(len(np.unique(x_labels)))]
        
#         self.x_labels = self.label_matrix(class_names, x_labels)

#         self.is_fitted_ = True

#         return self
    
#     def predict(self, Y):
#         # Y test sample
#         preds = []
        
#         # modify y's shape
#         y_s = np.zeros((Y.shape[1], Y.shape[0]))
#         for i in range(Y.shape[1]):
#             for j in range(Y.shape[0]):
#                 y_s[i,j] = Y[j,i]
        
#         xL2 = np.zeros(self.X.shape[1])
#         for i in range(self.X.shape[1]):
#             if np.random.rand() >= 0.5:
#                 xL2[i] = 1
#             else:
#                 xL2[i] = 0
                
#         for i in range(y_s.shape[0]):
#             reg = eknn.exclusive_lasso(self.X, y_s[i], self.group_vect, self.lambda_)
#             coefs = reg.lasso_optimize(xL2).x
#             knn_R = eknn.EkNN_R(self.X, y_s[i], coefs, self.x_labels, self.k)
#             preds.append(knn_R.predict())

#         return np.array(preds)
    
    
#     def label_matrix(self,label_names:list, labels):
#         class_num = len(label_names)
#         n_obs = labels.shape[0]
#         label_mat = np.zeros((class_num, n_obs))
#         for i in range(class_num):
#             for ind,j in enumerate(labels):
#                 if j == label_names[i]:
#                     label_mat[i,ind] = 1
#                 else:
#                     label_mat[i,ind] = 0
#         return label_mat
    
    
#     def group_encode(self,X):
#         # call before having X
#         n = X.shape[1]
#         group_pop = int(n/self.group_num)
#         group_vect = []
#         start = 0
#         end = group_pop
#         while end <= n:
#             temp = np.zeros(X.shape[1])
#             if end + group_pop > n:
#                 temp[start:n] = 1
#                 group_vect.append(temp)
#                 break
#             else: temp[start:end] = 1
#             start = end
#             end += group_pop
#             group_vect.append(temp)
#         return np.array(group_vect)

#     def score(self, Y, y_labels):
#         """
#         Calculate the accuracy of the classifier.
#         """
#         y_pred = self.predict(Y)
#         return accuracy_score(y_labels, y_pred)

In [29]:
y_true_labels = np.zeros(Y[-1].shape[0])
for i in range(len(y_true_labels)):
    if Y[-1][i] == 1:
        y_true_labels[i] = 0
    else:
        y_true_labels[i] = 1
y_true_labels

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1.])

In [30]:
param_grid = {
    'lambda_': [0.1, 0.01, 0.02],
    'group_num': [ 2, 3,5],
    'k': [3, 5, 7]
}

# Create the custom classifier instance
clf = eknn.ExclusiveLassoKNNClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search = model_selection.GridSearchCV(clf, param_grid, cv=5)  # You can change the scoring parameter
grid_search.fit(X_transposed, x_labels)# Use your training data

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_


/Users/vuongnguyen/anaconda3/envs/anaconda_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:821: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/vuongnguyen/anaconda3/envs/anaconda_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 810, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/vuongnguyen/anaconda3/envs/anaconda_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 527, in __call__
    return estimator.score(*args, **kwargs)
  File "/Users/vuongnguyen/Documents/summer_24/eknn.py", line 305, in score
    y_pred = self.predict(Y)
  File "/Users/vuongnguyen/Documents/summer_24/eknn.py", line 262, in predict
    coefs = reg.lasso_optimize(xL2).x
  File "/Users/vuongnguyen/Documents/summer_24/eknn.py", line 45, in lasso_optimize
    return minimize(func, x0, method='SLSQP')
  File 

In [31]:
best_params

{'group_num': 2, 'k': 3, 'lambda_': 0.1}

In [16]:
clf = eknn.ExclusiveLassoKNNClassifier(group_num=2, k=3, lambda_=0.1)
clf.fit(X_transposed, x_labels)
score = clf.score(Y,y_true_labels)
score

0.4473684210526316

#### Test on simulated data